In [1]:
import pandas as pd
import numpy as np
import itertools
import math

#Plotting libraries
#import matplotlib.pyplot as plt
#import seaborn as sns

#Web Scappinlibraries
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
from datetime import timedelta
from datetime import date
import time

#plt.style.use('seaborn-white')
#plt.style.use('fivethirtyeight')
#pd.plotting.register_matplotlib_converters()
#%matplotlib inline

#Model libraries


#from datetime import timedelta
from orbit.models.dlt import DLTMAP, DLTAggregated, DLTFull
#from orbit.diagnostics.plot import plot_predicted_data
#from orbit.diagnostics.plot import plot_predicted_components
 

import warnings
warnings.filterwarnings("ignore")


In [2]:
#Preparing  Start Date and End Date for fetching data from investing.com
today  = date.today()
#yesterday = today - timedelta(days = 1)
#enddate = time.mktime(yesterday.timetuple())
enddate = time.mktime(today.timetuple())
enddate = int(enddate)

starting_day_of_current_year = datetime.now().date().replace(month=1, day=1)
starting_day_of_current_year
stdate=time.mktime(starting_day_of_current_year.timetuple())
stdate=int(stdate)

In [3]:
url="https://in.investing.com/indices/s-p-cnx-nifty-historical-data?end_date={}&st_date={}".format(enddate,stdate)

In [4]:
url

'https://in.investing.com/indices/s-p-cnx-nifty-historical-data?end_date=1653762600&st_date=1640975400'

In [5]:
#Loading Chrome Driver to fetch data
driver = webdriver.Chrome()

In [6]:
driver.get(url)

In [7]:
#Get Page Content Data
content = driver.page_source
soup = BeautifulSoup(content)

In [8]:
#Loading Dates Column
date=[]
for a in soup.findAll('td', attrs={'class':'col-rowDate'}):
    date_txt=a.find('span', attrs={'class':'text'})
    date.append(date_txt.text)

In [9]:
#Loading Closing Prices
close=[]
for a in soup.findAll('td', attrs={'class':'col-last_close'}):
    close_txt=a.find('span', attrs={'class':'text'})
    close.append(close_txt.text)

In [10]:
#Loading Open Prices
open=[]
for a in soup.findAll('td', attrs={'class':'col-last_open'}):
    open_txt=a.find('span', attrs={'class':'text'})
    open.append(open_txt.text)

In [11]:
#Loading High Prices
high=[]
for a in soup.findAll('td', attrs={'class':'col-last_max'}):
    high_txt=a.find('span', attrs={'class':'text'})
    high.append(high_txt.text)

In [12]:
#Loading Low Prices
low=[]
for a in soup.findAll('td', attrs={'class':'col-last_min'}):
    low_txt=a.find('span', attrs={'class':'text'})
    low.append(low_txt.text)

In [13]:
## Prepare DataFrame

df_nifty = pd.DataFrame({'Date':date,'Open':open,'High':high,'Low':low,'Close':close}) 

In [14]:
df_nifty.head()

,Date,Open,High,Low,Close
0,"May 27, 2022","16,296.60","16,370.60","16,221.95","16,352.45"
1,"May 26, 2022","16,105.00","16,204.45","15,903.70","16,170.15"
2,"May 25, 2022","16,196.35","16,223.35","16,006.95","16,025.80"
3,"May 24, 2022","16,225.55","16,262.80","16,078.60","16,125.15"
4,"May 23, 2022","16,290.95","16,414.70","16,185.75","16,214.70"


In [15]:
df_nifty['Date'] = df_nifty['Date'].str.replace(r",","")

df_nifty['Date']=pd.to_datetime(df_nifty.Date , format = '%b %d %Y')

In [16]:
df_nifty=df_nifty.drop_duplicates(subset="Date")

In [17]:
#data = df_nifty.drop(['Date'], axis=1)
data = df_nifty

In [18]:
data['Close']=data['Close'].str.replace(r",","").astype('float')
data['Open']=data['Open'].str.replace(r",","").astype('float')
data['High']=data['High'].str.replace(r",","").astype('float')
data['Low']=data['Low'].str.replace(r",","").astype('float')
#data['Price']=data['Price'].astype('float')

In [19]:
data=data.fillna(method="ffill")

In [20]:
data.head()

,Date,Open,High,Low,Close
0,2022-05-27,16296.60,16370.60,16221.95,16352.45
1,2022-05-26,16105.00,16204.45,15903.70,16170.15
2,2022-05-25,16196.35,16223.35,16006.95,16025.80
3,2022-05-24,16225.55,16262.80,16078.60,16125.15
4,2022-05-23,16290.95,16414.70,16185.75,16214.70


In [21]:
data=data.sort_values(['Date'], ascending=[True])

In [22]:
data_log =    data[['Open', 'High', 'Low' , 'Close']].apply(np.log)
#data_log =    data[['Open', 'High', 'Low' , 'Price']].apply(np.log)

In [23]:
data_log.head()

,Open,High,Low,Close
98,9.763487,9.778301,9.763265,9.777113
97,9.780269,9.788503,9.775288,9.787249
96,9.788082,9.795050,9.784076,9.793966
95,9.785183,9.786839,9.778805,9.783910
94,9.786819,9.792835,9.781577,9.787667
